In [ ]:
# !open .

In [ ]:
import pandas as pd
import requests
import bs4
import math
import time
from datetime import date, timedelta
import NeoQuant as nq
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 1000)

## get all company list

In [ ]:
companies = nq.get_all_company_list()

## set company name

In [ ]:
# company_name = '현대건설'
company_name = '효성티앤씨'
company_code = nq.get_company_code(company_name, companies)
print('company name : ' + company_name)
print('company code : ' + company_code)

## get base profit ratio

In [ ]:
base_profit_ratio = nq.get_base_profit_ratio()
base_profit_ratio

## get fnguide snapshot

In [ ]:
if not company_code.startswith('A'):
    company_code = 'A' + company_code

snapshot_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701&gicode=' + company_code
snapshot_page = requests.get(snapshot_url)
snapshot_tables = pd.read_html(snapshot_page.text)

## get stock count

In [ ]:
import math

def get_total_stock_count(snapshot_tables):
    stock_count = 0

    info = snapshot_tables[0]
    info = info.set_index(info.columns[0])
    stock_count_info = info.loc['발행주식수(보통주/ 우선주)'][1]
    stock_counts = stock_count_info.split('/')
    for count in stock_counts:
#         print(count)
        stock_count = stock_count + int(count.replace(',',''))

    return stock_count

def get_self_stock_count(snapshot_tables):
    self_stock_count = 0
    info = snapshot_tables[4]
    info = info.set_index(info.columns[0])
    count = info.loc[['자기주식\xa0(자사주+자사주신탁)']]['보통주'][0]
#     print(count)
    if not math.isnan(count):
        self_stock_count = int(count)
    return self_stock_count

stock_count = get_total_stock_count(snapshot_tables) - get_self_stock_count(snapshot_tables)
print('stock count : ' + str(stock_count))

## get asset, roe

In [ ]:
def get_fr_df_index(snapshot_tables):
    fr_df_index = -1
    for num, snapshot in enumerate(snapshot_tables):
        if 'Annual' in snapshot.columns:
            if len(snapshot['Annual'].columns) > 4:
#                 print('bingo ' + str(num))
                fr_df_index = num
                break
    return fr_df_index

def get_base_year_name(fr_df):
    big_col_name = 'Annual'
    roe_index_name = 'ROE'
    asset_index_name = '지배주주지분'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    stadard_col_name = ''
    roes = info.loc[roe_index_name][big_col_name]
    assets = info.loc[asset_index_name][big_col_name]
    for index in reversed(assets.index):
        if index.endswith(('(E)')):
            continue
        if not math.isnan(assets[index]) and not math.isnan(roes[index]):
            stadard_col_name = index
            break
    return stadard_col_name

def get_year_name_list(fr_df, until_this_year = True):
    big_col_name = 'Annual'
    roe_index_name = 'ROE'
    asset_index_name = '지배주주지분'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    year_name_list = []
    
    year_name = ''
    roes = info.loc[roe_index_name][big_col_name]
    assets = info.loc[asset_index_name][big_col_name]
    today_year = datetime.date.today().year
    for index in assets.index:
#         if index.endswith(('(E)')):
#             continue
        if until_this_year:
            year = int(index[:4])
            if year > today_year:
                continue
        if not math.isnan(assets[index]) and not math.isnan(roes[index]):
            year_name = index
#             year_name = str(pd.to_datetime(year_name).year)
            year_name_list.append(year_name)
    return year_name_list

def get_roe(fr_df, base_year_name):
    big_col_name = 'Annual'
    roe_index_name = 'ROE'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    roe = info.loc[roe_index_name][big_col_name, base_year_name]
    return roe

def get_per(fr_df, base_year_name):
    big_col_name = 'Annual'
    per_index_name = 'PER'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    per = info.loc[per_index_name][big_col_name, base_year_name]
    return per

def get_net_income(fr_df, base_year_name):
    big_col_name = 'Annual'
    index_name = '당기순이익'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    income = info.loc[index_name][big_col_name, base_year_name]
    income = income * 100000000
    return income

def get_net_profit(fr_df, base_year_name):
    big_col_name = 'Annual'
    profit_index_name = '지배주주지분'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    profit = info.loc[profit_index_name][big_col_name, base_year_name]
    profit = profit * 100000000
    return profit

def get_op_profit(fr_df, base_year_name):
    big_col_name = 'Annual'
    profit_index_name = '영업이익'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    profit = info.loc[profit_index_name][big_col_name, base_year_name]
    profit = profit * 100000000
    return profit

def get_asset(fr_df, base_year_name):
    big_col_name = 'Annual'
    asset_index_name = '자산총계'
    
    info = fr_df
    info = info.set_index(info.columns[0])

    asset = info.loc[asset_index_name][big_col_name, base_year_name]
    asset = asset * 100000000
    return asset

In [ ]:
price_df_tmp = nq.request_price_list(company_code, 'day', 1500)

In [ ]:
price_df = price_df_tmp
price_df['price_very_low'] = 0
price_df['price_low'] = 0
price_df['price_middle'] = 0
price_df['price_high'] = 0

In [ ]:
fr_df_index = get_fr_df_index(snapshot_tables)
print('fr_df_index : ' + str(fr_df_index))
fr_df = snapshot_tables[fr_df_index]

In [ ]:
until_this_year = False
year_name_list = get_year_name_list(fr_df, until_this_year)

In [ ]:
final_year = price_df.iloc[-1].name.year

for year_name in year_name_list:
#     print(year_name)
    roe =  get_roe(fr_df, year_name)
    net_profit = get_net_profit(fr_df, year_name)
    per = get_per(fr_df, year_name)
    
    price_high = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 1)
    price_middle = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.9)
    price_low = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.8)
    price_very_low = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.5)
    

    print('=====================================')
    print('base year : ' + year_name)
    print('net profit : ' + str(net_profit))
    print('roe : ' + str(roe))
    print('per : ' + str(per))
    print('very low : ' + str(price_very_low))
    print('buy : below ' + str(price_low))
    print('sell 1/3 : ' + str(price_middle))
    print('sell 1/3 : ' + str(price_high))

    
#     if (price_very_low >= price_low):
#         price_low = int(price_very_low * 1.05)
#         price_middle = int(price_low * 1.05)
#         price_high = int(price_middle * 1.05)
        
#         print('======= modify ===============')
#         print('very low : ' + str(price_very_low))
#         print('buy : below ' + str(price_low))
#         print('sell 1/3 : ' + str(price_middle))
#         print('sell 1/3 : ' + str(price_high))
        
    if (year_name.endswith('(E)')):
#         continue
        year_name = year_name.replace('(E)', '')
    
    base_year = pd.to_datetime(year_name).year

    if (base_year == final_year):
        start_date = price_df.iloc[-1].name + timedelta(days=1)
        end_date = pd.to_datetime(str(final_year) + '-12-31')
        year_list = pd.date_range(start_date, end_date, freq='d')
        year_df = pd.DataFrame(0, index=year_list, columns=price_df.columns)
        price_df = price_df.append(year_df)
    
    if (base_year > final_year):
#         print(base_year)
        start_year = pd.to_datetime(str(base_year))
        year_list = pd.date_range(start_year, freq=pd.DateOffset(days=1), periods=365)
        year_df = pd.DataFrame(0, index=year_list, columns=price_df.columns)
        price_df = price_df.append(year_df)
        
    base_year = str(base_year)
    try:
        price_df.loc[base_year]['price_very_low'] = [price_very_low] * len(price_df.loc[base_year])
        price_df.loc[base_year]['price_low'] = [price_low] * len(price_df.loc[base_year])
        price_df.loc[base_year]['price_middle'] = [price_middle] * len(price_df.loc[base_year])
        price_df.loc[base_year]['price_high'] = [price_high] * len(price_df.loc[base_year])
    except KeyError:
        print('key error')

## show chart

In [ ]:
nq.show_price_chart(company_code, company_name, price_df)